In [1]:
import pandas as pd
import sqlite3

import sys
import os
# Modify path temporarily to import functions from NASEM_functions directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from NASEM_functions.ration_balancing_equations import *

In [2]:
# User will type the ration into input.txt
# This function reads that file and creates a dataframe with all of the feeds and % DM 

def read_input(input):
    data = []
    animal_input = {}

    with open(input, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith('#') or ':' not in line:
                continue

            if line.startswith('*'):
                key, value = line[1:].split(":")
                animal_input[key.strip()] = float(value.strip())
                continue

            feedstuff, per_DM = line.split(":")
            data.append([feedstuff.strip(), per_DM.strip()])

    user_input = pd.DataFrame(data, columns=["Feedstuff", "%_DM_user"])
    user_input['%_DM_user'] = user_input['%_DM_user'].astype(float)
    
    user_input['Feedstuff'] = user_input['Feedstuff'].str.strip()
    # user_input.set_index('Feedstuff', inplace=True)  # Set 'Feedstuff' as the index


    user_input['Index'] = user_input['Feedstuff']
    user_input = user_input.set_index('Index') 
    # user_input.index = user_input.index.str.strip()

    # user_input = user_input.set_index('Feedstuff')
    # user_input.set_index('Feedstuff', inplace=True)
    # user_input.index = user_input.index.str.strip()
    
    return user_input, animal_input

user_input, animal_input = read_input('input.txt')
list_of_feeds = user_input['Feedstuff'].tolist()
# list_of_feeds = user_input.index.tolist()

In [3]:
# Takes a list of the feed names and gets all their info from the feed library

def fl_get_rows(feeds_to_get):
    conn = sqlite3.connect('../../diet_database.db')
    cursor = conn.cursor()

    index_str = ', '.join([f"'{idx}'" for idx in feeds_to_get])

    query = f"SELECT * FROM NASEM_feed_library WHERE Fd_Name IN ({index_str})"

    cursor.execute(query)
    rows = cursor.fetchall()

    cursor.execute(f"PRAGMA table_info(NASEM_feed_library)")
    column_names = [column[1] for column in cursor.fetchall()]

    # Create a DataFrame from the retrieved rows with column names
    feed_data = pd.DataFrame(rows, columns=column_names)
    feed_data = feed_data.set_index('Fd_Name')
    feed_data.index = feed_data.index.str.strip()

    conn.close()

    return feed_data

feed_data = fl_get_rows(list_of_feeds)

In [4]:
# Get %DM to add to 100%
def set_perc_100():
    user_perc = user_input['%_DM_user'].sum()
    scaling_factor = 100 / user_perc

    user_input['%_DM_intake'] = user_input['%_DM_user'] * scaling_factor

    # Adjust so sum is exactly 100
    adjustment = 100 - user_input['%_DM_intake'].sum()
    user_input['%_DM_intake'] += adjustment / len(user_input)

    return user_input

user_input = set_perc_100()

In [5]:
# Predict DMI

def dmi_predicted(An_Parity_rl, Trg_MilkProd, An_BW, An_BCS, An_LactDay, Trg_MilkFatp, Trg_MilkTPp, Trg_MilkLacp):
    DMI = calculate_Dt_DMIn_Lact1(An_Parity_rl, Trg_MilkProd, An_BW, An_BCS, An_LactDay, Trg_MilkFatp, Trg_MilkTPp, Trg_MilkLacp)
    return DMI

animal_input['DMI'] = dmi_predicted(**animal_input)


In [6]:
# Get kg intake of each feed
# This may not be needed

def get_kg_intake(df, dict):
    DMI = dict['DMI']
    df['kg_intake'] = df['%_DM_intake']/100 * DMI

# get_kg_intake(user_input, animal_input)

In [58]:
# Old Calculate Nutrient Intakes
# This is an onlder method to calculate feed intakes

def get_nutrient_intakes_OLD(df):
    for feed in df['Feedstuff']:
        
        # CP Intake
        df.loc[df['Feedstuff'] == feed, 'CP_%_diet'] = feed_data.loc[feed, 'Fd_CP'] * df.loc[df['Feedstuff'] == feed, '%_DM_intake'] / 100
        # df.loc[df['Feedstuff'] == feed, 'CP_kg/d_diet'] = (feed_data.loc[feed, 'Fd_CP'] / 100) * df.loc[df['Feedstuff'] == feed, 'kg_intake']
        df.loc[df['Feedstuff'] == feed, 'CP_kg/d_diet'] = (df.loc[df['Feedstuff'] == feed, 'CP_%_diet'] / 100) * animal_input['DMI']
        # Alternate way to calculate kg/d, shoter to write

        # RUP Intake
        df.loc[df['Feedstuff'] == feed, 'RUP_%_CP'] = feed_data.loc[feed, 'Fd_RUP_base'] * df.loc[df['Feedstuff'] == feed, '%_DM_intake'] / 100
        df.loc[df['Feedstuff'] == feed, 'RUP_%_diet'] = df.loc[df['Feedstuff'] == feed, 'RUP_%_CP'] * feed_data.loc[feed, 'Fd_CP'] / 100

        # df.loc[df['Feedstuff'] == feed, 'RUP_kg/d_diet'] = (df.loc[df['Feedstuff'] == feed, 'RUP_%_CP'] / 100) * animal_input['DMI']


# get_nutrient_intakes(user_input)

In [9]:
# Rewrite get_nutrient_intakes to use a list of feed values to calculate, add these values to user_input dataframe then delete when calculations done
# Also make functions repeat for feeds that have the same calcultion (% DM)

def get_nutrient_intakes_dev(df):
    # List all the columns of interest here
    intakes_to_calculate = ['Fd_CP', 'Fd_RUP_base', 'Fd_NDF', 'Fd_ADF', 'Fd_St', 'Fd_CFat', 'Fd_Ash']
    
    # List any values that do not have the units % DM
    units_not_DM = ['Fd_RUP_base'] 

    for intake in intakes_to_calculate:
        
        if intake not in units_not_DM:
            df[intake] = df['Feedstuff'].map(feed_data[intake]) / 100                                # Get value from feed_data as a percentage

            df[intake + '_%_diet'] = df[intake] * df['%_DM_intake']                                  # Calculate component intake on %DM basis
            df[intake + '_kg/d_diet'] = df[intake + '_%_diet'] * animal_input['DMI'] / 100           # Calculate component kg intake 
        
        elif intake == 'Fd_RUP_base':                                                                # RUP is in % CP so an extra conversion is needed
            df[intake] = df['Feedstuff'].map(feed_data[intake]) / 100
            df['Fd_RUP_base_%_CP'] = df[intake] * df['%_DM_intake']
            df['Fd_RUP_base_%_diet'] = df['Fd_RUP_base_%_CP'] * df['Fd_CP']
            df['Fd_RUP_base_kg/d_diet'] = df['Fd_RUP_base_%_diet'] * animal_input['DMI'] / 100
    
    # Drop the columns with feed data
    df = df.drop(columns=intakes_to_calculate)

    return df


user_input = get_nutrient_intakes_dev(user_input)

['Fd_CP', 'Fd_RUP_base', 'Fd_NDF', 'Fd_ADF', 'Fd_St', 'Fd_CFat', 'Fd_Ash']


In [13]:
# Use this to compare feeds with NASEM 8 calculations
print(user_input['Fd_RUP_base_%_diet'].sum())
print(user_input['Fd_RUP_base_kg/d_diet'].sum())


6.647844563102679
1.6301445418620835
